<a href="https://colab.research.google.com/github/ninja-marduk/ml_precipitation_prediction/blob/feature%2Fhybrid-models/models/hybrid_models_enconders_layering_w3_ST-HybridWaveStack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meta-Modelo Convolucional: ElevClusConvPrecipMetaNet

Este modelo implementa una arquitectura convolucional avanzada para la predicción espaciotemporal de precipitación con horizonte de 12 meses.

## Arquitectura

1. **Entradas**:
   - Mapas de predicción de los modelos base (ConvBiGRU-AE y ConvLSTM-AE) para cada horizonte (12 meses)
   - Información de elevación y clusters para condicionamiento (FiLM)

2. **Reducción Temprana de Canales**:
   - Reduce los 24 canales (2 modelos × 12 horizontes) a 16 para optimizar memoria
   - Aplica Conv2D(1×1) para mezclar información sin perder resolución espacial

3. **Bloques Residuales Multiescala**:
   - Bloques depthwise-separables con distintas dilataciones (1,2,4)
   - Captura patrones a diferentes escalas espaciales sin incrementar parámetros

4. **Atención Espacial por Cluster**:
   - FiLM (Feature-wise Linear Modulation): γ_cluster ⊗ F + β_cluster
   - Adapta el comportamiento según el régimen orográfico

5. **U-Net Compacto**:
   - Arquitectura de encoder-decoder con skip connections
   - Solo 2 niveles de downsampling para preservar detalle

6. **Agrupamiento de Horizontes**:
   - Conv3D para procesar conjuntamente la dimensión temporal de horizontes
   - Permite aprender relaciones entre meses consecutivos

7. **Salida Multi-Horizonte**:
   - Genera los 12 mapas refinados de predicción

8. **Estrategias Memory-Friendly**:
   - Mixed precision (float16)
   - Gradient checkpointing
   - Acumulación de gradientes
   - Entrenamiento por etapas

In [ ]:
# Predicción Espaciotemporal de Precipitación Mensual - Notebook Completo

# 0) Configuración del entorno, rutas y dependencias
import sys
import os
import logging
from pathlib import Path
import joblib  # Para persistir scalers
import datetime

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

def log_and_print(msg):
    logger.info(msg)
    print(msg)

# Detectar entorno (Colab o local)
IN_COLAB = "google.colab" in sys.modules
log_and_print(f"Ejecutando en Colab: {IN_COLAB}")

# Definir rutas base
desired_repo = 'ml_precipitation_prediction'
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    BASE_PATH = Path('/content/drive/MyDrive') / desired_repo
    if not Path(desired_repo).exists():
        log_and_print("Clonando repositorio...")
        get_ipython().system('git clone https://github.com/ninja-marduk/ml_precipitation_prediction.git')
    os.chdir(desired_repo)
    get_ipython().system('pip install -q xarray netCDF4 optuna matplotlib seaborn lightgbm xgboost scikit-learn ace_tools_open cartopy geopandas joblib')
else:
    # Instalación más simple sin PyEMD
    %pip install -q xarray netCDF4 scikit-image
    current = Path.cwd()
    for p in [current] + list(current.parents):
        if (p / '.git').is_dir() or (p / 'requirements.txt').is_file() or (p / 'README.md').is_file():
            BASE_PATH = p
            break
    else:
        BASE_PATH = current
    log_and_print(f"Ejecutando en local. Base path: {BASE_PATH}")

# Rutas de datos y modelos
DATA_OUTPUT   = BASE_PATH / 'data' / 'output'
MODELS_OUTPUT = BASE_PATH / 'models' / 'output'
PREDS_DIR     = MODELS_OUTPUT / 'base_model_predictions'
SHP_PATH      = BASE_PATH / 'data' / 'input' / 'shapes' / 'MGN_Departamento.shp'

# Crear directorios
MODELS_OUTPUT.mkdir(parents=True, exist_ok=True)
PREDS_DIR.mkdir(parents=True, exist_ok=True)

# Parámetros generales
INPUT_WINDOW   = 60
OUTPUT_HORIZON = 12  # 12 meses
BATCH_SIZE     = 16
MAX_EPOCHS     = 300
PATIENCE       = 50
LR             = 1e-3

import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
log_and_print(f"Usando dispositivo: {DEVICE}")


# 1) Imports adicionales y utilidades
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature  # Añadido para características cartográficas
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler # Mixed precision training
from torch.utils.checkpoint import checkpoint # Gradient checkpointing
from torch.optim.lr_scheduler import ReduceLROnPlateau  # Añadido para learning rate adaptativo
from IPython.display import clear_output  # Para actualizar gráficos durante entrenamiento
import pywt
from scipy.signal import hilbert
from skimage.restoration import denoise_wavelet

# 2) Carga y preprocesamiento de datos - Versión simplificada

def load_and_preprocess_data():
    log_and_print("Cargando datos...")
    
    # Cargar datos completos
    ds_full = xr.open_dataset(DATA_OUTPUT / 'complete_dataset_with_features_with_clusters_elevation_with_windows.nc')
    log_and_print(f"Dataset completo cargado, dims: {ds_full.dims}")
    
    # Cargar componentes directamente de archivos específicos
    ds_ceemdan = xr.open_dataset(MODELS_OUTPUT / 'features_CEEMDAN.nc')
    log_and_print(f"Dataset CEEMDAN cargado, dims: {ds_ceemdan.dims}")
    log_and_print(f"Variables disponibles en CEEMDAN: {list(ds_ceemdan.data_vars.keys())}")
    
    ds_tvfemd = xr.open_dataset(MODELS_OUTPUT / 'features_TVFEMD.nc')
    log_and_print(f"Dataset TVF-EMD cargado, dims: {ds_tvfemd.dims}")
    log_and_print(f"Variables disponibles en TVF-EMD: {list(ds_tvfemd.data_vars.keys())}")
    
    # Cargar shapefile para visualizaciones
    gdf = gpd.read_file(SHP_PATH)
    if gdf.crs is None:
        gdf = gdf.set_crs(epsg=4326)
    elif gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)
    log_and_print("Shapefile cargado y CRS validado.")

    # Extraer información temporal
    times = ds_full.time.values.astype('datetime64[M]')
    REF = np.datetime64('2024-02','M')
    idx_ref = int(np.where(times==REF)[0][0])
    log_and_print(f"Referencia (REF) = {REF}, index={idx_ref}")
    
    return ds_full, ds_ceemdan, ds_tvfemd, gdf, times, REF, idx_ref

# Ejecutar carga/preproc con los archivos específicos
ds_full, ds_ceemdan, ds_tvfemd, gdf, times, REF, idx_ref = load_and_preprocess_data()

# Mantenemos sólo las funciones de preprocesamiento que aún se necesitan
def calculate_afc(signal, lags=[1, 3, 6]):
    """Calcula la Función de Autocorrelación para los lags dados."""
    afc = [np.correlate(signal[lag:], signal[:-lag], mode='valid') for lag in lags]
    return afc

# Funciones de preprocesamiento adicionales
def wavelet_denoise(data, wavelet='db4', level=3):
    """Aplica denoising wavelet a los datos."""
    try:
        # API moderna de scikit-image (0.19+)
        return denoise_wavelet(
            data, 
            wavelet=wavelet, 
            mode='soft',
            method='BayesShrink',
            channel_axis=None
        )
    except Exception as e:
        print(f"Error en wavelet denoising: {str(e)}. Intentando método alternativo.")
        # Si falla, intentar solo con parámetros básicos
        return denoise_wavelet(data, wavelet=wavelet)

# 3) Definición de Datasets PyTorch
class PrecipitationDataset(Dataset):
    """
    Dataset personalizado para datos de precipitación que maneja la reducción
    de dimensionalidad y garantiza la compatibilidad dimensional.
    """
    def __init__(self, data, target, seq_length):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).float()
        self.seq_length = seq_length
        self.target_shape = target.shape[1:]  # Guardar la forma objetivo (height, width)
        
        print(f"Dataset inicializado - Forma de datos: {self.data.shape}")
        print(f"Dataset inicializado - Forma de targets: {self.target.shape}")
        print(f"Forma objetivo almacenada: {self.target_shape}")

    def __len__(self):
        return len(self.data) - self.seq_length + 1

    def __getitem__(self, idx):
        # Obtener secuencia de datos
        inputs = self.data[idx:idx+self.seq_length]
        labels = self.target[idx:idx+self.seq_length]
        
        # Para debugging, imprimir formas solo para el primer elemento
        if idx == 0:
            print(f"Ejemplo de entrada - forma original: {inputs.shape}")
            
        # Reducir dimensionalidad de las características y transformar a tensor 2D
        if len(inputs.shape) == 2:  # [seq_length, features]
            # Crear un tensor 3D con forma [channels, height, width] 
            # con dimensiones espaciales adecuadas para redes convolucionales
            n_features = inputs.shape[1]
            feature_dim = min(int(np.sqrt(n_features)), 16)  # Limitar a un tamaño razonable
            
            # Seleccionar primeros feature_dim² características para crear mapa 2D
            n_features_to_use = min(feature_dim * feature_dim, n_features)
            flattened_features = inputs.mean(dim=0)[:n_features_to_use]
            
            # Reshape a forma [1, feature_dim, feature_dim] para canal único
            padding = torch.zeros(feature_dim * feature_dim - n_features_to_use) if n_features_to_use < feature_dim * feature_dim else None
            if padding is not None:
                flattened_features = torch.cat([flattened_features, padding])
            
            # Crear mapa 2D con canal único [1, H, W]
            spatial_features = flattened_features.reshape(1, feature_dim, feature_dim)
            inputs = spatial_features
            
        # Para debugging
        if idx == 0:
            print(f"Ejemplo de entrada - forma final: {inputs.shape}")
            print(f"Ejemplo de etiqueta: {labels.shape}")
            
        # CORREGIDO: Devolver sólo inputs y labels, omitir target_shape para compatibilidad
        return inputs, labels

# 4) Modelos Híbridos: ConvBiGRU-AE y ConvLSTM-AE
class ConvBiGRU_AE(nn.Module):
    def __init__(self, input_channels, hidden_dim, num_layers, output_channels, seq_length, target_shape=(61, 65), kernel_size=3, padding=1):
        super(ConvBiGRU_AE, self).__init__()
        self.seq_length = seq_length
        self.hidden_dim = hidden_dim
        self.output_channels = output_channels
        self.target_shape = target_shape  # Guardar la forma objetivo
        
        # Encoder - Convoluciones 2D para cada paso de tiempo
        # En lugar de esperar múltiples canales, procesaremos cada paso de tiempo independientemente
        self.conv1 = nn.Conv2d(1, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.norm1 = nn.InstanceNorm2d(hidden_dim)
        self.conv2 = nn.Conv2d(hidden_dim, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.norm2 = nn.InstanceNorm2d(hidden_dim)
        
        # GRU para procesar la secuencia comprimida
        self.gru = nn.GRU(hidden_dim, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True)
        
        # Decoder
        self.deconv1 = nn.ConvTranspose2d(hidden_dim * 2, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.dnorm1 = nn.InstanceNorm2d(hidden_dim)
        self.deconv2 = nn.ConvTranspose2d(hidden_dim, output_channels * seq_length, kernel_size=kernel_size, padding=padding)
        
    def forward(self, x, target_shape=None):
        # Usar la forma objetivo proporcionada o la predeterminada
        if target_shape is None:
            target_shape = self.target_shape
        
        # x tiene forma [batch_size, seq_length, height, width]
        batch_size, seq_len, H, W = x.size()
        
        # Procesar cada paso de tiempo individualmente
        processed_features = []
        
        for t in range(seq_len):
            # Obtener el paso de tiempo actual y añadir dimensión de canal
            x_t = x[:, t].unsqueeze(1)  # [batch_size, 1, height, width]
            
            # Aplicar convoluciones
            x_t = F.relu(self.norm1(self.conv1(x_t)))
            x_t = F.relu(self.norm2(self.conv2(x_t)))
            
            # Extraer características globales
            # Promedio espacial para reducir a [batch_size, hidden_dim]
            x_t_features = x_t.mean(dim=(2, 3))
            
            # Almacenar características para este paso de tiempo
            processed_features.append(x_t_features)
        
        # Concatenar características para todos los pasos de tiempo
        sequence_features = torch.stack(processed_features, dim=1)  # [batch_size, seq_length, hidden_dim]
        
        # Aplicar GRU a la secuencia
        output, _ = self.gru(sequence_features)  # [batch_size, seq_length, hidden_dim*2]
        
        # Tomar el último estado y preparar para deconv
        output = output[:, -1, :].view(batch_size, self.hidden_dim*2, 1, 1)
        output = F.interpolate(output, size=(H, W), mode='nearest')
        
        # Decoder
        output = F.relu(self.dnorm1(self.deconv1(output)))
        output = self.deconv2(output)  # [batch_size, seq_length*output_channels, H, W]
        
        # Reorganizar para obtener [batch_size, seq_length, output_channels, H, W]
        output = output.view(batch_size, self.seq_length, self.output_channels, H, W)
        
        # Redimensionar a la forma objetivo usando interpolación bilineal
        # Primero reorganizamos para [batch_size*seq_length, output_channels, H, W]
        output_reshaped = output.view(batch_size * self.seq_length, self.output_channels, H, W)
        output_resized = F.interpolate(output_reshaped, size=target_shape, mode='bilinear', align_corners=False)
        
        # Reorganizar de nuevo a [batch_size, seq_length, output_channels, target_H, target_W]
        output = output_resized.view(batch_size, self.seq_length, self.output_channels, target_shape[0], target_shape[1])
        
        return output

class ConvBiLSTM_AE(nn.Module):
    def __init__(self, input_channels, hidden_dim, num_layers, output_channels, seq_length, target_shape=(61, 65), kernel_size=3, padding=1):
        super(ConvBiLSTM_AE, self).__init__()
        self.seq_length = seq_length
        self.hidden_dim = hidden_dim
        self.output_channels = output_channels
        self.target_shape = target_shape  # Guardar la forma objetivo
        
        # Encoder para procesar cada paso de tiempo individualmente
        self.conv1 = nn.Conv2d(1, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.norm1 = nn.InstanceNorm2d(hidden_dim)
        self.conv2 = nn.Conv2d(hidden_dim, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.norm2 = nn.InstanceNorm2d(hidden_dim)
        
        # LSTM para procesar la secuencia
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True)
        
        # Decoder
        self.deconv1 = nn.ConvTranspose2d(hidden_dim * 2, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.dnorm1 = nn.InstanceNorm2d(hidden_dim)
        self.deconv2 = nn.ConvTranspose2d(hidden_dim, output_channels * seq_length, kernel_size=kernel_size, padding=padding)
        
    def forward(self, x, target_shape=None):
        # Usar la forma objetivo proporcionada o la predeterminada
        if target_shape is None:
            target_shape = self.target_shape
            
        # x tiene forma [batch_size, seq_length, height, width]
        batch_size, seq_len, H, W = x.size()
        
        # Procesar cada paso de tiempo individualmente
        processed_features = []
        
        for t in range(seq_len):
            # Obtener el paso de tiempo actual y añadir dimensión de canal
            x_t = x[:, t].unsqueeze(1)  # [batch_size, 1, height, width]
            
            # Aplicar convoluciones
            x_t = F.relu(self.norm1(self.conv1(x_t)))
            x_t = F.relu(self.norm2(self.conv2(x_t)))
            
            # Extraer características globales
            # Promedio espacial para reducir a [batch_size, hidden_dim]
            x_t_features = x_t.mean(dim=(2, 3))
            
            # Almacenar características para este paso de tiempo
            processed_features.append(x_t_features)
        
        # Concatenar características para todos los pasos de tiempo
        sequence_features = torch.stack(processed_features, dim=1)  # [batch_size, seq_length, hidden_dim]
        
        # Aplicar LSTM a la secuencia
        output, _ = self.lstm(sequence_features)  # [batch_size, seq_length, hidden_dim*2]
        
        # Tomar el último estado y preparar para deconv
        output = output[:, -1, :].view(batch_size, self.hidden_dim*2, 1, 1)
        output = F.interpolate(output, size=(H, W), mode='nearest')
        
        # Decoder
        output = F.relu(self.dnorm1(self.deconv1(output)))
        output = self.deconv2(output)  # [batch_size, seq_length*output_channels, H, W]
        
        # Reorganizar para obtener [batch_size, seq_length, output_channels, H, W]
        output = output.view(batch_size, self.seq_length, self.output_channels, H, W)
        
        # Redimensionar a la forma objetivo usando interpolación bilineal
        output_reshaped = output.view(batch_size * self.seq_length, self.output_channels, H, W)
        output_resized = F.interpolate(output_reshaped, size=target_shape, mode='bilinear', align_corners=False)
        output = output_resized.view(batch_size, self.seq_length, self.output_channels, target_shape[0], target_shape[1])
        
        return output

# 5) Funciones de entrenamiento y evaluación
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, num_epochs=500, patience=50):
    """
    Entrena el modelo usando los loaders proporcionados, con soporte para scheduler y más métricas.
    Compatible con diferentes versiones de PyTorch.
    """
    train_losses, val_losses = [], []
    best_val_loss = float('inf')
    epochs_no_improve = 0
    
    # Guardar ruta para el modelo
    model_path = MODELS_OUTPUT / 'best_model_temp.pth'
    
    # Verificar la versión de PyTorch para usar los parámetros adecuados al guardar modelos
    import torch
    torch_version = torch.__version__
    print(f"Versión de PyTorch detectada: {torch_version}")
    
    # Función para guardar modelo compatible con diferentes versiones de PyTorch
    def save_model(model, path):
        try:
            torch.save(model.state_dict(), path, weights_only=True)
            print("Modelo guardado con parámetro weights_only=True")
        except TypeError:
            try:
                torch.save(model.state_dict(), path, _use_new_zipfile_serialization=True)
                print("Modelo guardado con parámetro _use_new_zipfile_serialization=True")
            except TypeError:
                torch.save(model.state_dict(), path)
                print("Modelo guardado sin parámetros adicionales")
    
    # Función para cargar modelo compatible con diferentes versiones
    def load_model(model, path):
        try:
            model.load_state_dict(torch.load(path, weights_only=True))
            print("Modelo cargado con parámetro weights_only=True")
        except TypeError:
            try:
                model.load_state_dict(torch.load(path, map_location=DEVICE))
                print("Modelo cargado con parámetro map_location")
            except TypeError:
                model.load_state_dict(torch.load(path))
                print("Modelo cargado sin parámetros adicionales")
    
    # Para graficar el progreso durante el entrenamiento
    def plot_progress():
        clear_output(wait=True)
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.axhline(y=best_val_loss, color='r', linestyle='--', label=f'Best: {best_val_loss:.2f}')
        plt.title(f'Loss vs. Epochs (Current: {val_losses[-1]:.2f})')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(alpha=0.3)
        
        plt.subplot(1, 2, 2)
        rel_loss = [l/train_losses[0] for l in train_losses]
        rel_val_loss = [l/val_losses[0] for l in val_losses]
        plt.plot(rel_loss, label='Train')
        plt.plot(rel_val_loss, label='Val')
        plt.title(f'Relative Loss (% of initial loss)')
        plt.xlabel('Epoch')
        plt.ylabel('Relative Loss')
        plt.legend()
        plt.grid(alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    for epoch in range(num_epochs):
        model.train()
        batch_train_losses = []
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            
            # Debug info en primera iteración
            if epoch == 0 and len(batch_train_losses) == 0:
                print(f"Batch entrenamiento - inputs: {inputs.shape}, targets: {targets.shape}")
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            # Asegurar que outputs y targets tienen formas comparables para calcular la pérdida
            if len(outputs.shape) == 5 and len(targets.shape) <= 4:
                outputs = outputs.squeeze(2)  # eliminar dim C si es 1
                
            if len(outputs.shape) != len(targets.shape):
                if len(outputs.shape) == 5 and len(targets.shape) == 3:
                    targets = targets.unsqueeze(1).unsqueeze(2).repeat(1, outputs.shape[1], 1, 1, 1)
                elif len(outputs.shape) == 5 and len(targets.shape) == 4:
                    targets = targets.unsqueeze(2)
            
            loss = criterion(outputs, targets)
            loss.backward()
            # Gradient clipping para estabilidad
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            batch_train_losses.append(loss.item())
        
        train_loss = np.mean(batch_train_losses)
        train_losses.append(train_loss)
        
        model.eval()
        batch_val_losses = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                outputs = model(inputs)
                
                # Ajustar dimensiones si es necesario
                if len(outputs.shape) == 5 and len(targets.shape) <= 4:
                    outputs = outputs.squeeze(2)
                
                if len(outputs.shape) != len(targets.shape):
                    if len(outputs.shape) == 5 and len(targets.shape) == 3:
                        targets = targets.unsqueeze(1).unsqueeze(2).repeat(1, outputs.shape[1], 1, 1, 1)
                    elif len(outputs.shape) == 5 and len(targets.shape) == 4:
                        targets = targets.unsqueeze(2)
                
                loss = criterion(outputs, targets)
                batch_val_losses.append(loss.item())
        
        val_loss = np.mean(batch_val_losses)
        val_losses.append(val_loss)
        
        # Usar scheduler si está disponible
        if scheduler is not None:
            scheduler.step(val_loss)
        
        # Mostrar progreso cada 10 épocas o en la última
        if epoch % 10 == 0 or epoch == num_epochs - 1:
            print(f'Epoca {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
            plot_progress()  # Actualizar el gráfico
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            # Usar la función save_model compatible con diferentes versiones
            save_model(model, model_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print('Early stopping triggered')
                break
    
    # Cargar el mejor modelo con la función load_model compatible
    load_model(model, model_path)
    return model, train_losses, val_losses

# También necesitamos corregir la función al guardar los modelos finales al final del entrenamiento
def save_model_compatible(model, path):
    """
    Guarda un modelo de manera compatible con diferentes versiones de PyTorch.
    """
    try:
        torch.save(model.state_dict(), path, weights_only=True)
    except TypeError:
        try:
            torch.save(model.state_dict(), path, _use_new_zipfile_serialization=True)
        except TypeError:
            torch.save(model.state_dict(), path)
    print(f"Modelo guardado en {path}")

def save_model(model, path, epoch=0, val_loss=0.0):
    """
    Guarda un modelo PyTorch con metadatos, de forma compatible con diferentes versiones PyTorch.
    
    Args:
        model: Modelo PyTorch a guardar
        path: Ruta donde guardar el modelo
        epoch: Número de época actual
        val_loss: Pérdida de validación actual
    """
    try:
        # Crear directorio si no existe
        os.makedirs(os.path.dirname(path), exist_ok=True)
        
        # Guardar modelo con metadatos
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'epoch': epoch,
            'val_loss': val_loss
        }
        
        # Intentar diferentes métodos de guardado según compatibilidad de versión
        try:
            torch.save(checkpoint, path, weights_only=True)
        except TypeError:
            try:
                torch.save(checkpoint, path, _use_new_zipfile_serialization=True)
            except TypeError:
                torch.save(checkpoint, path)
        
        print(f"Modelo guardado en {path}")
    except Exception as e:
        print(f"Error al guardar el modelo en {path}: {str(e)}")

# Visualización mejorada con coordenadas geográficas
def visualize_predictions_with_geospatial_coords():
    """
    Visualiza las predicciones usando coordenadas geoespaciales reales
    y muestra un mapa con más detalle, incluyendo límites administrativos.
    """
    try:
        import os
        
        # Determinar si tenemos archivos de predicciones
        pred_file = PREDS_DIR / 'convbigru_predictions.npy'
        if not os.path.exists(pred_file):
            print("No se encontraron archivos de predicciones. Generando predicciones...")
            
            # Generar predicciones si no existen archivos
            model = convbigru_ae
            model.eval()
            
            # Usar un solo batch del dataset de validación
            val_batch = next(iter(val_loader))
            inputs, targets = val_batch
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            
            # Hacer predicción
            with torch.no_grad():
                outputs = model(inputs)
                
            # Convertir a numpy
            predictions = outputs.cpu().detach().numpy()
            targets_np = targets.cpu().detach().numpy()
            
            # Guardar para uso futuro
            np.save(PREDS_DIR / 'convbigru_predictions.npy', predictions)
            np.save(PREDS_DIR / 'targets.npy', targets_np)
            
            print(f"Predicciones generadas y guardadas con forma: {predictions.shape}")
            
            # Usar las predicciones generadas
            if len(predictions.shape) == 5:
                # [B, seq, C, H, W]
                sample_pred = predictions[0, 0, 0]  # Primer batch, primer elemento de secuencia, primer canal
            elif len(predictions.shape) == 4:
                # [B, seq, H, W]
                sample_pred = predictions[0, 0]     # Primer batch, primer elemento de secuencia
            else:
                sample_pred = predictions
                
            # Mismo proceso para targets
            if len(targets_np.shape) == 4:
                sample_target = targets_np[0, 0]    # Primer batch, primer elemento de secuencia
            elif len(targets_np.shape) == 3:
                sample_target = targets_np[0]       # Primer batch
            else:
                sample_target = targets_np
        else:
            # Cargar predicciones existentes
            predictions = np.load(PREDS_DIR / 'convbigru_predictions.npy')
            targets = np.load(PREDS_DIR / 'targets.npy')
            
            # Extraer muestra para visualización
            if len(predictions.shape) == 5:
                sample_pred = predictions[0, 0, 0]  # [B, seq, C, H, W] -> [H, W]
            elif len(predictions.shape) == 4:
                sample_pred = predictions[0, 0]     # [B, seq, H, W] -> [H, W]
            else:
                sample_pred = predictions
                
            if len(targets.shape) == 4:
                sample_target = targets[0, 0]       # [B, seq, H, W] -> [H, W]
            elif len(targets.shape) == 3:
                sample_target = targets[0]          # [B, H, W] -> [H, W]
            else:
                sample_target = targets
        
        # Extraer coordenadas lat/lon del dataset
        latitudes = ds_full.latitude.values
        longitudes = ds_full.longitude.values
        
        # Crear malla de coordenadas
        lon_mesh, lat_mesh = np.meshgrid(longitudes, latitudes)
        
        # Crear figura con proyección de mapa
        fig = plt.figure(figsize=(20, 10))
        
        # Definir proyección y límites del mapa para Colombia/región de interés
        projection = ccrs.PlateCarree()
        
        # Crear tres subfiguras con la misma proyección
        ax1 = fig.add_subplot(131, projection=projection)
        ax2 = fig.add_subplot(132, projection=projection)
        ax3 = fig.add_subplot(133, projection=projection)
        
        # Configurar cada subplot
        for ax, data, title in zip([ax1, ax2, ax3], 
                                   [sample_target, sample_pred, sample_pred - sample_target], 
                                   ['Valores Reales', 'Predicción ConvBiGRU-AE', 'Error (Predicción - Real)']):
            # Añadir características del mapa
            ax.coastlines(resolution='10m', color='black', linewidth=1)
            ax.add_feature(cfeature.BORDERS, linestyle=':')
            ax.add_feature(cfeature.LAKES, alpha=0.5)
            
            # Añadir el shapefile de Colombia
            ax.add_geometries(gdf.geometry, crs=ccrs.PlateCarree(), edgecolor='black',
                             facecolor='none', alpha=0.8, linewidth=0.5)
            
            # Crear mapa de contorno con coordenadas reales
            im = ax.pcolormesh(lon_mesh, lat_mesh, data, cmap='viridis', 
                              transform=ccrs.PlateCarree())
            
            # Añadir barra de color
            cbar = fig.colorbar(im, ax=ax, orientation='horizontal', pad=0.05, fraction=0.05)
            cbar.ax.tick_params(labelsize=8)
            
            # Añadir título y cuadrícula
            ax.set_title(title, fontsize=14)
            gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)
            gl.top_labels = False
            gl.right_labels = False
            
            # Establecer límites del mapa para la zona de Boyacá en Colombia
            ax.set_extent([-75.5, -71.5, 4.0, 7.5], crs=ccrs.PlateCarree())
        
        plt.tight_layout()
        plt.savefig(MODELS_OUTPUT / 'geospatial_predictions.png', dpi=300, bbox_inches='tight')
        plt.show()
        
    except Exception as e:
        print(f"Error al visualizar predicciones geoespaciales: {str(e)}")
        import traceback
        traceback.print_exc()

# 6) Preparación de datos para modelos híbridos - Versión simplificada

# Función para generar predicciones del modelo base en caso de que no existan
def create_base_model_predictions(ds_full, idx_ref):
    """
    Entrena un modelo ConvBiGRU básico y genera predicciones para usar como entrada
    del meta-modelo, para evitar el uso de datos sintéticos.
    """
    print("\n" + "="*70)
    print("GENERACIÓN DE PREDICCIONES CON MODELO BASE ConvBiGRU")
    print("="*70)
    
    print("\n🔄 Iniciando generación de predicciones del modelo base...")
    
    # Verificar si ya existen las predicciones en el dataset
    if 'convbigru_preds' in ds_full.data_vars:
        print("✅ Las predicciones 'convbigru_preds' ya existen en el dataset.")
        return ds_full
    
    # 1. Seleccionar datos para entrenamiento y validación
    print("1️⃣ Preparando datos para modelo base...")
    
    # Usaremos precipitation como entrada y target
    if 'total_precipitation' in ds_full:
        precip_var = 'total_precipitation'
    elif 'precip' in ds_full:
        precip_var = 'precip'
    else:
        # Buscar cualquier variable que contenga 'precip' en el nombre
        precip_vars = [var for var in ds_full.data_vars if 'precip' in var.lower()]
        if precip_vars:
            precip_var = precip_vars[0]
        else:
            print("❌ No se encontró ninguna variable de precipitación")
            return ds_full
    
    print(f"   - Usando variable '{precip_var}' como entrada y objetivo")
    
    # Separar en train y validación usando la fecha de referencia
    ds_base_train = ds_full.sel(time=slice(None, ds_full.time.values[idx_ref-1]))
    ds_base_val = ds_full.sel(time=slice(ds_full.time.values[idx_ref], None))
    
    # Convertir a arrays NumPy
    X_base_train = ds_base_train[precip_var].values.astype(np.float32)
    y_base_train = X_base_train.copy()  # Mismo input/output para el modelo base
    
    X_base_val = ds_base_val[precip_var].values.astype(np.float32)
    y_base_val = X_base_val.copy()
    
    # Añadir dimensión de canal si es necesario
    if len(X_base_train.shape) == 3:  # [tiempo, lat, lon]
        X_base_train = X_base_train.reshape(X_base_train.shape[0], 1, X_base_train.shape[1], X_base_train.shape[2])
        X_base_val = X_base_val.reshape(X_base_val.shape[0], 1, X_base_val.shape[1], X_base_val.shape[2])
        
    print(f"   - Forma de datos de entrenamiento: {X_base_train.shape}")
    print(f"   - Forma de datos de validación: {X_base_val.shape}")
    
    # 2. Crear dataset de PyTorch
    print("\n2️⃣ Creando datasets y dataloaders...")
    
    seq_length = min(12, X_base_train.shape[0] // 10)  # Secuencia más corta para modelo base
    
    # Convertir a tensores PyTorch
    X_base_train_tensor = torch.from_numpy(X_base_train).float()
    y_base_train_tensor = torch.from_numpy(y_base_train).float()
    
    X_base_val_tensor = torch.from_numpy(X_base_val).float()
    y_base_val_tensor = torch.from_numpy(y_base_val).float()
    
    # Crear datasets personalizados para secuencias
    class SimpleSeqDataset(Dataset):
        def __init__(self, features, targets, seq_length=12):
            self.features = features
            self.targets = targets
            self.seq_length = seq_length
            
        def __len__(self):
            return len(self.features) - self.seq_length + 1
            
        def __getitem__(self, idx):
            # Input: secuencia de 'seq_length' elementos
            x = self.features[idx:idx + self.seq_length]
            # Target: siguiente elemento después de la secuencia
            # Si queremos predecir múltiples pasos, podemos usar:
            y = self.targets[idx + self.seq_length - 1:idx + self.seq_length]
            return x, y
    
    # Crear datasets
    train_base_dataset = SimpleSeqDataset(X_base_train_tensor, y_base_train_tensor, seq_length)
    val_base_dataset = SimpleSeqDataset(X_base_val_tensor, y_base_val_tensor, seq_length)
    
    # Crear dataloaders
    batch_size_base = min(batch_size, len(train_base_dataset) // 10)  # Batch más pequeño si hay pocos datos
    batch_size_base = max(1, batch_size_base)  # Asegurar batch_size mínimo 1
    
    train_base_loader = DataLoader(train_base_dataset, batch_size=batch_size_base, shuffle=True)
    val_base_loader = DataLoader(val_base_dataset, batch_size=batch_size_base, shuffle=False)
    
    print(f"   - Longitud de secuencia: {seq_length}")
    print(f"   - Batch size: {batch_size_base}")
    print(f"   - Batches por época: {len(train_base_loader)}")
    
    # 3. Crear modelo base simplificado
    print("\n3️⃣ Creando modelo base ConvBiGRU simplificado...")
    
    input_channels_base = X_base_train.shape[1]
    hidden_dim_base = 64  # Más pequeño para modelo base
    output_channels_base = 1
    
    # Modelo ConvBiGRU simplificado para generar predicciones base
    class SimpleConvBiGRU(nn.Module):
        def __init__(self, input_channels, hidden_dim, output_channels):
            super(SimpleConvBiGRU, self).__init__()
            self.conv = nn.Conv2d(input_channels, hidden_dim, kernel_size=3, padding=1)
            self.bn = nn.BatchNorm2d(hidden_dim)
            self.gru = nn.GRU(hidden_dim, hidden_dim, bidirectional=True, batch_first=True)
            self.output_conv = nn.Conv2d(hidden_dim*2, output_channels, kernel_size=3, padding=1)
            
        def forward(self, x):
            # x tiene forma [batch, sequence, channels, height, width]
            batch_size, seq_len, C, H, W = x.shape
            
            # Procesar cada paso de tiempo
            outputs = []
            for t in range(seq_len):
                # Obtener el frame actual
                x_t = x[:, t]  # [batch, channels, height, width]
                
                # Aplicar convolución
                x_t = F.relu(self.bn(self.conv(x_t)))
                
                # Extraer características para GRU (promedio espacial)
                features = x_t.view(batch_size, -1, H*W)
                features = features.mean(dim=2)  # [batch, hidden_dim]
                
                # Añadir dimensión de secuencia para GRU
                features = features.unsqueeze(1)  # [batch, 1, hidden_dim]
                
                # Si es el primer paso, inicializar salida GRU
                if t == 0:
                    gru_out, h = self.gru(features)
                else:
                    gru_out, h = self.gru(features, h)
                
                # Reformar para conv final
                gru_features = gru_out.view(batch_size, -1, 1, 1)  # [batch, hidden_dim*2, 1, 1]
                gru_features = F.interpolate(gru_features, size=(H, W), mode='bilinear', align_corners=False)
                
                # Generar salida
                out = self.output_conv(gru_features)  # [batch, output_channels, height, width]
                outputs.append(out)
            
            # Concatenar todos los outputs
            return torch.stack(outputs, dim=1)  # [batch, sequence, output_channels, height, width]
    
    # Instanciar modelo
    base_model = SimpleConvBiGRU(input_channels_base, hidden_dim_base, output_channels_base).to(DEVICE)
    print(f"   - Modelo creado con {input_channels_base} canales de entrada, {hidden_dim_base} dimensiones ocultas")
    
    # 4. Entrenar modelo
    print("\n4️⃣ Entrenando modelo base...")
    
    # Hiperparámetros
    lr_base = 1e-3
    epochs_base = 30
    patience_base = 10
    
    # Optimizador y función de pérdida
    criterion_base = nn.MSELoss()
    optimizer_base = optim.Adam(base_model.parameters(), lr=lr_base)
    
    # Entrenamiento
    best_val_loss = float('inf')
    epochs_no_improve = 0
    
    for epoch in range(epochs_base):
        # Train
        base_model.train()
        train_losses = []
        
        for batch_idx, (data, target) in enumerate(train_base_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            
            # Forward pass
            optimizer_base.zero_grad()
            output = base_model(data)
            
            # Ajustar dimensiones para calcular pérdida
            if output.shape != target.shape:
                if len(output.shape) == 5 and len(target.shape) == 4:
                    output = output.squeeze(2)  # Eliminar dimensión de canal si es 1
                elif len(output.shape) == 5 and len(target.shape) == 3:
                    output = output[:, -1, 0]   # Tomar último elemento de secuencia y primer canal
            
            # Calcular pérdida
            loss = criterion_base(output, target)
            train_losses.append(loss.item())
            
            # Backward pass
            loss.backward()
            optimizer_base.step()
        
        # Validation
        base_model.eval()
        val_losses = []
        
        with torch.no_grad():
            for data, target in val_base_loader:
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = base_model(data)
                
                # Ajustar dimensiones
                if output.shape != target.shape:
                    if len(output.shape) == 5 and len(target.shape) == 4:
                        output = output.squeeze(2)
                    elif len(output.shape) == 5 and len(target.shape) == 3:
                        output = output[:, -1, 0]
                
                # Calcular pérdida
                loss = criterion_base(output, target)
                val_losses.append(loss.item())
        
        # Calculamos pérdidas promedio
        avg_train_loss = sum(train_losses) / len(train_losses)
        avg_val_loss = sum(val_losses) / len(val_losses)
        
        # Mostrar progreso
        print(f"   Época {epoch+1}/{epochs_base}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # Guardar mejor modelo
            torch.save(base_model.state_dict(), MODELS_OUTPUT / 'simple_convbigru_base.pth')
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve >= patience_base:
            print("   Early stopping - No mejora en validación durante varias épocas")
            break
    
    # 5. Generar predicciones para todo el dataset
    print("\n5️⃣ Generando predicciones para todo el dataset...")
    
    # Cargar mejor modelo
    base_model.load_state_dict(torch.load(MODELS_OUTPUT / 'simple_convbigru_base.pth'))
    base_model.eval()
    
    # Crear dataset para todas las fechas
    all_data = torch.from_numpy(X_base_train).float()
    if len(X_base_val) > 0:
        all_data = torch.cat([all_data, torch.from_numpy(X_base_val).float()], dim=0)
    
    # Generar predicciones
    all_predictions = []
    seq_len = seq_length
    
    with torch.no_grad():
        for i in range(0, len(all_data) - seq_len + 1):
            # Obtener secuencia
            seq = all_data[i:i+seq_len].unsqueeze(0).to(DEVICE)  # Add batch dimension
            
            # Generar predicción
            pred = base_model(seq)
            
            # Si estamos en el último paso, añadimos las últimas predicciones
            if i == 0:
                # Añadir todas las predicciones de la primera secuencia
                for j in range(seq_len):
                    if len(pred.shape) == 5:  # [B, seq, C, H, W]
                        single_pred = pred[0, j, 0].cpu().numpy()  # Primera batch, paso j, canal 0
                    else:
                        single_pred = pred[0, j].cpu().numpy()  # Primera batch, paso j
                    all_predictions.append(single_pred)
            else:
                # Para el resto, añadir solo la última predicción (paso a paso)
                if len(pred.shape) == 5:  # [B, seq, C, H, W]
                    single_pred = pred[0, -1, 0].cpu().numpy()  # Primera batch, último paso, canal 0
                else:
                    single_pred = pred[0, -1].cpu().numpy()  # Primera batch, último paso
                all_predictions.append(single_pred)
    
    # Convertir lista de predicciones a array
    predictions_array = np.array(all_predictions)
    print(f"   - Forma de predicciones generadas: {predictions_array.shape}")
    
    # Verificar que tenemos predicciones para todas las fechas
    if len(predictions_array) < len(ds_full.time):
        print(f"⚠️ No se generaron suficientes predicciones ({len(predictions_array)} vs {len(ds_full.time)} fechas)")
        missing = len(ds_full.time) - len(predictions_array)
        # Rellenar con los últimos valores repetidos
        last_pred = predictions_array[-1]
        for _ in range(missing):
            predictions_array = np.concatenate([predictions_array, last_pred[np.newaxis, ...]], axis=0)
    
    # 6. Guardar predicciones en el dataset
    print("\n6️⃣ Guardando predicciones en el dataset...")
    
    # Añadir variable al dataset
    ds_updated = ds_full.copy()
    ds_updated['convbigru_preds'] = (('time', 'latitude', 'longitude'), predictions_array)
    
    # Guardar dataset actualizado
    output_file = DATA_OUTPUT / 'dataset_with_convbigru_preds.nc'
    ds_updated.to_netcdf(output_file)
    print(f"✅ Dataset con predicciones guardado en {output_file}")
    
    print("\n✅ Proceso completo: modelo entrenado y predicciones generadas")
    return ds_updated

def prepare_data_for_hybrid_models(ds_full, ds_ceemdan, ds_tvfemd, idx_ref, horizon=12):
    """
    Prepara los datos para el entrenamiento de los modelos híbridos ConvBiGRU-AE y ConvLSTM-AE.
    Con verificación de límites para evitar IndexError y cálculos robustos de fechas.
    
    Args:
        ds_full: Dataset completo con todas las características.
        ds_ceemdan: Dataset con características de CEEMDAN.
        ds_tvfemd: Dataset con características de TVF-EMD.
        idx_ref: Índice de la fecha de referencia en el dataset.
        horizon: Horizonte de predicción (número de meses).
        
    Returns:
        X_train, y_train, X_val, y_val: Conjuntos de datos preparados para entrenamiento y validación.
    """
    # Verificar límites y ajustar índices si es necesario
    max_idx = len(ds_full.time.values) - 1
    
    # Verificar que hay suficientes datos para validación
    if idx_ref + horizon > max_idx:
        available_horizon = max_idx - idx_ref
        print(f"⚠️ Advertencia: No hay suficientes datos futuros. Ajustando horizonte de {horizon} a {available_horizon}.")
        horizon = available_horizon
    
    # Para datos de entrenamiento, usar todo hasta referencia (excepto último mes para tener targets disponibles)
    # Asegurarnos de que idx_ref > 0 para evitar errores
    if idx_ref <= 0:
        raise ValueError(f"El índice de referencia ({idx_ref}) debe ser mayor que 0.")
    
    # Obtener las fechas para los conjuntos de entrenamiento y validación
    train_end_date = ds_full.time.values[idx_ref]
    val_end_idx = min(idx_ref + horizon, max_idx)
    
    print(f"📅 Rango de fechas: entrenamiento hasta {train_end_date}, validación hasta {ds_full.time.values[val_end_idx]}")
    print(f"📊 Horizonte efectivo: {horizon} meses")
    
    # Para entrenamiento, usar datos históricos hasta la fecha de referencia (exclusive)
    ds_train = ds_full.sel(time=slice(None, ds_full.time.values[idx_ref-1]))
    
    # Para validación, usar datos desde la fecha de referencia hasta el horizonte disponible
    ds_val = ds_full.sel(time=slice(ds_full.time.values[idx_ref], ds_full.time.values[val_end_idx]))
    
    print(f"📏 Tamaño conjunto de entrenamiento: {len(ds_train.time)}")
    print(f"📏 Tamaño conjunto de validación: {len(ds_val.time)}")
    
    # VALIDACIÓN DE DATOS:
    # 1. Verificar si tenemos suficientes datos
    if len(ds_train.time) < 12:
        print("⚠️ ADVERTENCIA: Muy pocos datos para entrenamiento (<12 meses)!")
    if len(ds_val.time) < 3:
        print("⚠️ ADVERTENCIA: Muy pocos datos para validación (<3 meses)!")
        
    # 2. Verificar que los conjuntos no se superpongan
    train_times = set(ds_train.time.values.astype('datetime64[M]').astype(str))
    val_times = set(ds_val.time.values.astype('datetime64[M]').astype(str))
    if train_times.intersection(val_times):
        print("⚠️ ERROR: Superposición entre conjuntos de entrenamiento y validación!")
        
    # 3. Verificar variables disponibles
    available_vars = list(ds_train.data_vars.keys())
    print(f"📋 Variables disponibles: {available_vars[:5]}... (total: {len(available_vars)})")
    
    # Extraer características y etiquetas
    # Primero verificar si tenemos predictores precomputados
    use_synthetic_data = False
    if 'convbigru_preds' in ds_train:
        print("✅ Usando predicciones precomputadas 'convbigru_preds'")
        X_train = ds_train['convbigru_preds'].values.astype(np.float32)
        X_val = ds_val['convbigru_preds'].values.astype(np.float32)
    else:
        # No tenemos predicciones, usamos datos sintéticos
        use_synthetic_data = True
        print("ℹ️ Variable 'convbigru_preds' no encontrada. Preparando datos alternativos...")
        
        # Intentar usar precipitación directamente
        if 'total_precipitation' in ds_train:
            print("✅ Usando 'total_precipitation' como característica principal")
            precipitation_var = 'total_precipitation'
        elif 'precip' in ds_train:
            print("✅ Usando 'precip' como característica principal")
            precipitation_var = 'precip'
        else:
            # Intentar encontrar algo relacionado con precipitación
            precip_vars = [var for var in available_vars if 'precip' in var.lower()]
            if precip_vars:
                precipitation_var = precip_vars[0]
                print(f"✅ Usando '{precipitation_var}' como característica principal")
            else:
                # Usar la primera variable disponible
                precipitation_var = available_vars[0]
                print(f"⚠️ No se encontraron variables de precipitación. Usando '{precipitation_var}'")
        
        # Preparar datos usando la variable seleccionada
        X_train = ds_train[precipitation_var].values.astype(np.float32)
        X_val = ds_val[precipitation_var].values.astype(np.float32)
        
        # Añadir dimensión de canal si es necesario [tiempo, lat, lon] -> [tiempo, 1, lat, lon]
        if len(X_train.shape) == 3:
            X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2])
            X_val = X_val.reshape(X_val.shape[0], 1, X_val.shape[1], X_val.shape[2])
    
    # Target es siempre precipitación
    if 'total_precipitation' in ds_train:
        y_train = ds_train['total_precipitation'].values.astype(np.float32)
        y_val = ds_val['total_precipitation'].values.astype(np.float32)
    elif 'precip' in ds_train:
        y_train = ds_train['precip'].values.astype(np.float32)
        y_val = ds_val['precip'].values.astype(np.float32)
    else:
        # Si no encontramos precipitación, usar la misma variable que para las características
        y_train = X_train.copy()
        y_val = X_val.copy()
        # Si tiene dimensión de canal, la quitamos para el target
        if len(y_train.shape) == 4:
            y_train = y_train.squeeze(1)
            y_val = y_val.squeeze(1)
    
    # Imprimir información sobre los datos
    print(f"\n📊 Resumen de los datos:")
    print(f"  - X_train: {X_train.shape}, Rango: [{X_train.min():.2f}, {X_train.max():.2f}]")
    print(f"  - y_train: {y_train.shape}, Rango: [{y_train.min():.2f}, {y_train.max():.2f}]")
    print(f"  - X_val: {X_val.shape}, Rango: [{X_val.min():.2f}, {X_val.max():.2f}]")
    print(f"  - y_val: {y_val.shape}, Rango: [{y_val.min():.2f}, {y_val.max():.2f}]")
    
    # Recordatorio final
    if use_synthetic_data:
        print("\n⚠️ NOTA: Se están usando datos sintéticos porque no se encontró 'convbigru_preds'")
        print("   Si esto es inesperado, verifique que el dataset incluye las variables necesarias.")
    
    return X_train, y_train, X_val, y_val

# Verificar si necesitamos generar predicciones base
if 'convbigru_preds' not in ds_full.data_vars:
    print("\n🚩 No se encontraron predicciones 'convbigru_preds'. Generando modelo base y predicciones...")
    ds_full = create_base_model_predictions(ds_full, idx_ref)
    print("\n✅ Ahora puede continuar con el entrenamiento del modelo meta usando las predicciones generadas.")
else:
    print("\n✅ El dataset ya contiene las predicciones 'convbigru_preds'. No es necesario generarlas.")

# Aplicar la función mejorada
print("\n🔄 Preparando datos con la función mejorada...")
X_train, y_train, X_val, y_val = prepare_data_for_hybrid_models(ds_full, ds_ceemdan, ds_tvfemd, idx_ref, horizon=OUTPUT_HORIZON)

# Verificar formas de los conjuntos de datos
print(f"\n✅ Formas finales de los conjuntos de datos:")
print(f"  - Entrenamiento (X): {X_train.shape}")
print(f"  - Entrenamiento (y): {y_train.shape}")
print(f"  - Validación (X): {X_val.shape}")
print(f"  - Validación (y): {y_val.shape}")

# Configuración de hiperparámetros mejorados para entrenamiento
print("\n\n----- CONFIGURACIÓN DE HIPERPARÁMETROS MEJORADOS -----")

# Parámetros del modelo mejorados
input_channels = X_train.shape[1]  # Número de características de entrada
hidden_dim = 128  # Aumentado de 64 a 128 para mayor capacidad
num_layers = 3    # Aumentado de 2 a 3 para mayor profundidad
output_channels = 1
seq_length = OUTPUT_HORIZON  # Definimos seq_length como igual al horizonte de predicción
learning_rate = 0.0005  # Reducido para una convergencia más estable
num_epochs = 500  # Aumentado sustancialmente de 200 a 500
patience = 50     # Aumentado para permitir más intentos antes de early stopping
batch_size = 16   # Mantenemos el mismo tamaño de batch

# Inicializar modelos con arquitectura más potente
print(f"Inicializando modelos con input_channels={input_channels}, hidden_dim={hidden_dim}...")
convbigru_ae = ConvBiGRU_AE(input_channels, hidden_dim, num_layers, output_channels, seq_length).to(DEVICE)
convbilstm_ae = ConvBiLSTM_AE(input_channels, hidden_dim, num_layers, output_channels, seq_length).to(DEVICE)

# Optimizadores con decay para evitar sobreajuste
criterion = nn.MSELoss()
optimizer_convbigru = optim.Adam(convbigru_ae.parameters(), lr=learning_rate, weight_decay=1e-5)
optimizer_convbilstm = optim.Adam(convbilstm_ae.parameters(), lr=learning_rate, weight_decay=1e-5)

# Añadir schedulers para reducir el learning rate cuando la pérdida se estanca
scheduler_convbigru = ReduceLROnPlateau(optimizer_convbigru, mode='min', factor=0.5, patience=20, verbose=True)
scheduler_convbilstm = ReduceLROnPlateau(optimizer_convbilstm, mode='min', factor=0.5, patience=20, verbose=True)

# Create dataset objects with the prepared data
train_dataset = PrecipitationDataset(X_train, y_train, seq_length)
val_dataset = PrecipitationDataset(X_val, y_val, seq_length)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Configuración actualizada: {num_epochs} épocas, LR={learning_rate}, paciencia={patience}")

# Verificar compatibilidad con forward pass
# Probar un forward pass con un batch del DataLoader
try:
    sample_inputs, sample_targets = next(iter(train_loader))
    sample_inputs = sample_inputs.to(DEVICE)
    sample_targets = sample_targets.to(DEVICE)
    
    # Probar con ambos modelos
    with torch.no_grad():
        sample_outputs_bigru = convbigru_ae(sample_inputs)
        sample_outputs_bilstm = convbilstm_ae(sample_inputs)
    
    print("Forward pass exitoso. Formas de salida:")
    print(f"  - ConvBiGRU-AE: {sample_outputs_bigru.shape}")
    print(f"  - ConvBiLSTM-AE: {sample_outputs_bilstm.shape}")
    models_ready = True
except Exception as e:
    print(f"Error durante el forward pass de prueba: {str(e)}")
    models_ready = False

# Entrenamiento con las funciones mejoradas y mayor número de épocas
if models_ready:
    try:
        print("\n" + "="*50)
        print("ENTRENAMIENTO DE CONVBIGRU_AE")
        print("="*50)
        convbigru_ae, train_losses_bigru, val_losses_bigru = train_model(
            convbigru_ae, train_loader, val_loader, criterion, 
            optimizer_convbigru, scheduler_convbigru,
            num_epochs, patience
        )
        
        print("\n" + "="*50)
        print("ENTRENAMIENTO DE CONVBILSTM_AE")
        print("="*50)
        convbilstm_ae, train_losses_bilstm, val_losses_bilstm = train_model(
            convbilstm_ae, train_loader, val_loader, criterion, 
            optimizer_convbilstm, scheduler_convbilstm,
            num_epochs, patience
        )
        
        print("\n✅ Entrenamiento completado correctamente")
        
        # Guardar modelos entrenados
        save_model_compatible(convbigru_ae, MODELS_OUTPUT / 'convbigru_ae_model.pth')
        save_model_compatible(convbilstm_ae, MODELS_OUTPUT / 'convbilstm_ae_model.pth')
        print(f"Modelos guardados en {MODELS_OUTPUT}")
        
        # Evaluación y generación de visualizaciones geoespaciales
        print("\n" + "="*50)
        print("VISUALIZACIÓN DE PREDICCIONES CON COORDENADAS GEOESPACIALES")
        print("="*50)
        
        # Generar visualizaciones geoespaciales precisas
        visualize_predictions_with_geospatial_coords()
        
        # También visualizar las curvas de aprendizaje detalladas
        def plot_detailed_learning_curves():
            """
            Crea visualizaciones detalladas de las curvas de aprendizaje para los modelos
            ConvBiGRU-AE y ConvBiLSTM-AE, incluyendo análisis de convergencia y comparativa.
            """
            try:
                plt.figure(figsize=(20, 12))
                
                # 1. Gráfico de pérdida absoluta para ambos modelos
                plt.subplot(2, 2, 1)
                plt.plot(train_losses_bigru, label='ConvBiGRU - Train', color='blue', linestyle='-')
                plt.plot(val_losses_bigru, label='ConvBiGRU - Val', color='blue', linestyle='--')
                plt.plot(train_losses_bilstm, label='ConvBiLSTM - Train', color='red', linestyle='-')
                plt.plot(val_losses_bilstm, label='ConvBiLSTM - Val', color='red', linestyle='--')
                plt.title('Curvas de Aprendizaje - Pérdida Absoluta', fontsize=14)
                plt.xlabel('Época', fontsize=12)
                plt.ylabel('Pérdida (MSE)', fontsize=12)
                plt.legend(loc='upper right')
                plt.grid(alpha=0.3)
                
                # 2. Gráfico de pérdida relativa (normalizada al valor inicial)
                plt.subplot(2, 2, 2)
                rel_train_bigru = [l/train_losses_bigru[0] for l in train_losses_bigru]
                rel_val_bigru = [l/val_losses_bigru[0] for l in val_losses_bigru]
                rel_train_bilstm = [l/train_losses_bilstm[0] for l in train_losses_bilstm]
                rel_val_bilstm = [l/val_losses_bilstm[0] for l in val_losses_bilstm]
                
                plt.plot(rel_train_bigru, label='ConvBiGRU - Train', color='blue', alpha=0.7)
                plt.plot(rel_val_bigru, label='ConvBiGRU - Val', color='blue', linestyle='--', alpha=0.7)
                plt.plot(rel_train_bilstm, label='ConvBiLSTM - Train', color='red', alpha=0.7)
                plt.plot(rel_val_bilstm, label='ConvBiLSTM - Val', color='red', linestyle='--', alpha=0.7)
                plt.title(f'Curvas de Aprendizaje - Pérdida Relativa (% del valor inicial)', fontsize=14)
                plt.xlabel('Época', fontsize=12)
                plt.ylabel('Pérdida Relativa', fontsize=12)
                plt.legend()
                plt.grid(alpha=0.3)
                
                # 3. Comparación de diferencia entre train y validation
                plt.subplot(2, 2, 3)
                diff_bigru = [t-v for t, v in zip(train_losses_bigru, val_losses_bigru)]
                diff_bilstm = [t-v for t, v in zip(train_losses_bilstm, val_losses_bilstm)]
                
                plt.plot(diff_bigru, label='ConvBiGRU (Train-Val)', color='blue')
                plt.plot(diff_bilstm, label='ConvBiLSTM (Train-Val)', color='red')
                plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
                plt.title('Diferencia entre Pérdidas de Train y Validación', fontsize=14)
                plt.xlabel('Época', fontsize=12)
                plt.ylabel('Train Loss - Val Loss', fontsize=12)
                plt.legend()
                plt.grid(alpha=0.3)
                
                # 4. Tasa de mejora (derivada de la pérdida)
                plt.subplot(2, 2, 4)
                # Calcular mejora por época (primera derivada de la pérdida)
                improve_rate_bigru = [train_losses_bigru[i-1] - train_losses_bigru[i] for i in range(1, len(train_losses_bigru))]
                improve_rate_bilstm = [train_losses_bilstm[i-1] - train_losses_bilstm[i] for i in range(1, len(train_losses_bilstm))]
                
                plt.plot(improve_rate_bigru, label='ConvBiGRU', color='blue')
                plt.plot(improve_rate_bilstm, label='ConvBiLSTM', color='red')
                plt.title('Tasa de Mejora por Época (Δ Pérdida)', fontsize=14)
                plt.xlabel('Época', fontsize=12)
                plt.ylabel('Mejora (Reducción de Pérdida)', fontsize=12)
                plt.grid(alpha=0.3)
                plt.legend()
                
                # Ajustar diseño y guardar figura
                plt.tight_layout()
                plt.savefig(MODELS_OUTPUT / 'detailed_learning_curves.png', dpi=300, bbox_inches='tight')
                plt.show()
                
                # Resumen final de métricas
                best_val_bigru = min(val_losses_bigru)
                best_val_bilstm = min(val_losses_bilstm)
                best_epoch_bigru = val_losses_bigru.index(best_val_bigru)
                best_epoch_bilstm = val_losses_bilstm.index(best_val_bilstm)
                
                print("\n==== RESUMEN DE MÉTRICAS DE ENTRENAMIENTO ====")
                print(f"ConvBiGRU-AE:")
                print(f"  - Mejor pérdida de validación: {best_val_bigru:.2f} (Época {best_epoch_bigru})")
                print(f"  - Reducción total de pérdida: {train_losses_bigru[0] - train_losses_bigru[-1]:.2f} ({(1 - train_losses_bigru[-1]/train_losses_bigru[0])*100:.1f}%)")
                
                print(f"\nConvBiLSTM-AE:")
                print(f"  - Mejor pérdida de validación: {best_val_bilstm:.2f} (Época {best_epoch_bilstm})")
                print(f"  - Reducción total de pérdida: {train_losses_bilstm[0] - train_losses_bilstm[-1]:.2f} ({(1 - train_losses_bilstm[-1]/train_losses_bilstm[0])*100:.1f}%)")
                
                if best_val_bigru < best_val_bilstm:
                    print(f"\n✅ ConvBiGRU-AE tiene mejor rendimiento con {(best_val_bilstm - best_val_bigru)/best_val_bilstm*100:.1f}% menor pérdida de validación")
                else:
                    print(f"\n✅ ConvBiLSTM-AE tiene mejor rendimiento con {(best_val_bigru - best_val_bilstm)/best_val_bigru*100:.1f}% menor pérdida de validación")
                
            except Exception as e:
                print(f"Error al visualizar curvas de aprendizaje: {str(e)}")
                import traceback
                traceback.print_exc()
                
        plot_detailed_learning_curves()
        
    except Exception as e:
        print(f"\n❌ Error durante el entrenamiento: {str(e)}")
        import traceback
        traceback.print_exc()
else:
    print("\n❌ Entrenamiento cancelado debido a problemas con los modelos.")

# Implementación de Modelos Híbridos Avanzados

## TopoClus-CEEMDAN-TVF-AFC-ConvBiGRU‐AE y TopoClus-CEEMDAN-TVF-AFC-ConvLSTM‐AE
"""
Estos modelos representan una arquitectura avanzada que combina:

1. **Técnicas de descomposición de señales**:
   - CEEMDAN (Complete Ensemble Empirical Mode Decomposition with Adaptive Noise)
   - TVF-EMD (Time-Varying Filter Empirical Mode Decomposition)
   
2. **Características de autocorrelación (AFC)** en diferentes lags temporales

3. **Información topográfica y orográfica**:
   - Clusters basados en elevación (TopoClus)
   - Embeddings específicos por cluster

4. **Arquitecturas neuronales avanzadas**:
   - Encoder-Decoder Convolucional con BiGRU o BiLSTM
   - Atención topográfica para modular características por tipo de terreno

# Preparación de datos y extracción de características para modelos híbridos
"""

import numpy as np
import pandas as pd
import xarray as xr
from sklearn.preprocessing import StandardScaler
from pathlib import Path
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.checkpoint import checkpoint
import math

class FiLMLayer(nn.Module):
    """
    Feature-wise Linear Modulation para adaptar las características según el cluster orográfico
    """
    def __init__(self, n_clusters, n_features):
        super().__init__()
        self.embedding = nn.Embedding(n_clusters, n_features * 2)
        
        # Inicializar con valores razonables (gamma cercano a 1, beta cercano a 0)
        nn.init.normal_(self.embedding.weight[:, :n_features], 1.0, 0.1)
        nn.init.zeros_(self.embedding.weight[:, n_features:])
    
    def forward(self, x, cluster_idx):
        # x: [batch, channels, height, width]
        # cluster_idx: [batch]
        batch_size, channels = x.shape[0], x.shape[1]
        
        # Obtener parámetros gamma y beta del embedding
        params = self.embedding(cluster_idx)  # [batch, channels*2]
        gamma, beta = params.chunk(2, dim=1)  # [batch, channels], [batch, channels]
        
        # Reshape para permitir broadcasting
        gamma = gamma.view(batch_size, channels, 1, 1)
        beta = beta.view(batch_size, channels, 1, 1)
        
        # Aplicar modulación: γ ⊗ x + β
        return gamma * x + beta

class MultiResBranch(nn.Module):
    """
    Rama de procesamiento multi-resolución con dilataciones variables
    """
    def __init__(self, in_channels, out_channels, dilations=(1, 2, 4)):
        super().__init__()
        self.branches = nn.ModuleList()
        
        # Crear una rama para cada dilatación
        for dilation in dilations:
            branch = nn.Sequential(
                nn.Conv2d(
                    in_channels, 
                    out_channels, 
                    kernel_size=3, 
                    padding=dilation, 
                    dilation=dilation
                ),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
            )
            self.branches.append(branch)
    
    def forward(self, x):
        outputs = []
        for branch in self.branches:
            outputs.append(branch(x))
        
        # Concatenar resultados de todas las ramas
        return torch.cat(outputs, dim=1)

class TopoClus_CEEMDAN_TVF_AFC_Encoder(nn.Module):
    """
    Encoder compartido para ambos modelos que integra todas las fuentes de datos
    """
    def __init__(self, input_channels, hidden_dim, n_clusters, use_checkpoint=True):
        super().__init__()
        self.use_checkpoint = use_checkpoint
        
        # Reducción inicial de canales
        self.channel_reduction = nn.Conv2d(input_channels, hidden_dim, kernel_size=1)
        
        # Procesamiento multi-resolución
        self.multi_res = MultiResBranch(hidden_dim, hidden_dim//2)
        merged_channels = hidden_dim//2 * 3  # 3 ramas con dilaciones diferentes
        
        # Adaptación por cluster (FiLM)
        self.film = FiLMLayer(n_clusters, merged_channels)
        
        # Codificador principal (estructura tipo U-Net)
        self.down1 = nn.Sequential(
            nn.Conv2d(merged_channels, hidden_dim, 3, padding=1),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(),
            nn.Conv2d(hidden_dim, hidden_dim, 3, padding=1),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU()
        )
        
        self.pool1 = nn.MaxPool2d(2)
        
        self.down2 = nn.Sequential(
            nn.Conv2d(hidden_dim, hidden_dim*2, 3, padding=1),
            nn.BatchNorm2d(hidden_dim*2),
            nn.ReLU(),
            nn.Conv2d(hidden_dim*2, hidden_dim*2, 3, padding=1),
            nn.BatchNorm2d(hidden_dim*2),
            nn.ReLU()
        )
        
        self.pool2 = nn.MaxPool2d(2)
        
        self.bottleneck = nn.Sequential(
            nn.Conv2d(hidden_dim*2, hidden_dim*4, 3, padding=1),
            nn.BatchNorm2d(hidden_dim*4),
            nn.ReLU(),
            nn.Conv2d(hidden_dim*4, hidden_dim*4, 3, padding=1),
            nn.BatchNorm2d(hidden_dim*4),
            nn.ReLU()
        )
    
    def forward(self, x, cluster_idx):
        # x: [batch, channels, height, width]
        
        # Reducción de canales
        x = self.channel_reduction(x)
        
        # Procesamiento multiresolución
        if self.use_checkpoint and self.training:
            x = checkpoint(self.multi_res, x)
        else:
            x = self.multi_res(x)
        
        # Adaptación por cluster
        x = self.film(x, cluster_idx)
        
        # Codificador U-Net
        # Guardar para conexiones skip
        if self.use_checkpoint and self.training:
            x1 = checkpoint(self.down1, x)
        else:
            x1 = self.down1(x)
        
        x = self.pool1(x1)
        
        if self.use_checkpoint and self.training:
            x2 = checkpoint(self.down2, x)
        else:
            x2 = self.down2(x)
        
        x = self.pool2(x2)
        
        if self.use_checkpoint and self.training:
            x = checkpoint(self.bottleneck, x)
        else:
            x = self.bottleneck(x)
        
        return x, x1, x2  # Retornar también activaciones intermedias para skip connections

class TopoClus_CEEMDAN_TVF_AFC_ConvBiGRU_AE(nn.Module):
    """
    Modelo completo que integra codificador compartido con BiGRU para procesamiento temporal
    """
    def __init__(self, input_channels, hidden_dim, n_clusters, seq_length=12, output_channels=1):
        super().__init__()
        
        self.output_horizon = seq_length  # Guardar el horizonte de salida
        
        self.encoder = TopoClus_CEEMDAN_TVF_AFC_Encoder(
            input_channels, hidden_dim, n_clusters
        )
        
        # BiGRU para procesamiento secuencial
        self.bigru = nn.GRU(
            hidden_dim*4*4*4,  # Tamaño del bottleneck (asumiendo 2 maxpoolings)
            hidden_dim*4,
            bidirectional=True,
            batch_first=True,
            num_layers=2
        )
        
        # Decoder
        self.up1 = nn.ConvTranspose2d(hidden_dim*8, hidden_dim*2, 2, stride=2)
        self.dec1 = nn.Sequential(
            nn.Conv2d(hidden_dim*4, hidden_dim*2, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim*2),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden_dim*2, hidden_dim*2, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim*2),
            nn.ReLU(inplace=True)
        )
    
        self.up2 = nn.ConvTranspose2d(hidden_dim*2, hidden_dim, 2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(hidden_dim*2, hidden_dim, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, padding=1, bias=False), 
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(inplace=True)
        )
        
        # 7. Capa de salida multi-horizonte
        out_channels = self.output_horizon * output_channels
        self.output_conv = nn.Conv2d(hidden_dim, out_channels, kernel_size=1)
        
    def forward(self, x, cluster_idx, target_shape=None):
        batch_size = x.shape[0]
        
        # Codificar
        bottleneck, skip1, skip2 = self.encoder(x, cluster_idx)
        
        # Aplanar el bottleneck para el GRU
        flattened = bottleneck.view(batch_size, -1)
        
        # Reshape para BiGRU (añadir dim de secuencia)
        gru_in = flattened.unsqueeze(1)
        
        # Procesar con BiGRU
        gru_out, _ = self.bigru(gru_in)
        
        # Tomar salida y reshape para decodificador
        gru_features = gru_out.view(batch_size, -1, 1, 1)
        
        # Redimensionar para que coincida con el bottleneck
        h, w = bottleneck.shape[2], bottleneck.shape[3]
        gru_features = F.interpolate(gru_features, size=(h, w), mode='bilinear', align_corners=False)
        
        # Decodificar
        x = self.up1(gru_features)
        
        # Skip connection 1
        x = torch.cat([x, skip2], dim=1)
        x = self.dec1(x)
        
        x = self.up2(x)
        
        # Skip connection 2
        x = torch.cat([x, skip1], dim=1)
        x = self.dec2(x)
        
        # Salida multi-horizonte
        x = self.output_conv(x)
        
        # Si no se proporciona target_shape, usar la forma de x
        if target_shape is None:
            height, width = x.shape[2], x.shape[3]  # Usar forma de la salida actual
            # Ajustar para output_channels y output_horizon
            target_height = height // self.output_horizon
            target_width = width // output_channels
            target_shape = (target_height, target_width)
        
        # Reorganizar para obtener [batch, seq, channels, height, width]
        output = x.reshape(batch_size, self.output_horizon, output_channels, target_shape[0], target_shape[1])
        
        return output
        
class TopoClus_CEEMDAN_TVF_AFC_ConvBiLSTM_AE(nn.Module):
    """
    Versión con BiLSTM en lugar de BiGRU
    """
    def __init__(self, input_channels, hidden_dim, n_clusters, seq_length=12, output_channels=1):
        super().__init__()
        
        self.output_horizon = seq_length  # Guardar el horizonte de predicción
        
        self.encoder = TopoClus_CEEMDAN_TVF_AFC_Encoder(
            input_channels, hidden_dim, n_clusters
        )
        
        # BiLSTM para procesamiento secuencial
        self.bilstm = nn.LSTM(
            hidden_dim*4*4*4,  # Tamaño del bottleneck (asumiendo 2 maxpoolings)
            hidden_dim*4,
            bidirectional=True,
            batch_first=True,
            num_layers=2
        )
        
        # Decodificador (mismo que en versión BiGRU)
        self.up1 = nn.ConvTranspose2d(hidden_dim*8, hidden_dim*2, 2, stride=2)
        self.dec1 = nn.Sequential(
            nn.Conv2d(hidden_dim*4, hidden_dim*2, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim*2),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden_dim*2, hidden_dim*2, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim*2),
            nn.ReLU(inplace=True)
        )
        
        self.up2 = nn.ConvTranspose2d(hidden_dim*2, hidden_dim, 2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(hidden_dim*2, hidden_dim, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, padding=1, bias=False), 
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(inplace=True)
        )
        
        # 7. Capa de salida multi-horizonte
        out_channels = self.output_horizon * output_channels
        self.output_conv = nn.Conv2d(hidden_dim, out_channels, kernel_size=1)
        
    def forward(self, x, cluster_idx, target_shape=None):
        batch_size = x.shape[0]
        
        # Codificar
        bottleneck, skip1, skip2 = self.encoder(x, cluster_idx)
        
        # Aplanar el bottleneck para el LSTM
        flattened = bottleneck.view(batch_size, -1)
        
        # Reshape para BiLSTM (añadir dim de secuencia)
        lstm_in = flattened.unsqueeze(1)
        
        # Procesar con BiLSTM
        lstm_out, _ = self.bilstm(lstm_in)
        
        # Tomar salida y reshape para decodificador
        lstm_features = lstm_out.view(batch_size, -1, 1, 1)
        
        # Redimensionar para que coincida con el bottleneck
        h, w = bottleneck.shape[2], bottleneck.shape[3]
        lstm_features = F.interpolate(lstm_features, size=(h, w), mode='bilinear', align_corners=False)
        
        # Decodificar (mismo proceso que BiGRU)
        x = self.up1(lstm_features)
        x = torch.cat([x, skip2], dim=1)
        x = self.dec1(x)
        
        x = self.up2(x)
        x = torch.cat([x, skip1], dim=1)
        x = self.dec2(x)
        
        # Salida multi-horizonte
        x = self.output_conv(x)
        
        # Si no se proporciona target_shape, usar la forma de x
        if target_shape is None:
            height, width = x.shape[2], x.shape[3]  # Usar forma de la salida actual
            # Ajustar para output_channels y output_horizon
            target_height = height // self.output_horizon
            target_width = width // output_channels
            target_shape = (target_height, target_width)
            
        # Reorganizar para obtener [batch, seq, channels, height, width]
        output = x.reshape(batch_size, self.output_horizon, output_channels, target_shape[0], target_shape[1])
        
        return output

# Dataset para manejar las múltiples fuentes de características
class MultiSourceDataset(Dataset):
    """
    Dataset personalizado para manejar múltiples fuentes de características
    """
    def __init__(self, X_list, y, seq_length=12):
        """
        Args:
            X_list: Lista de arrays de características
            y: Array de targets
            seq_length: Longitud de la secuencia
        """
        self.X_list = [torch.FloatTensor(x) for x in X_list]
        self.y = torch.FloatTensor(y)
        self.seq_length = seq_length
        
        # Definir nombres de características basados en la estructura de X_list
        feature_names = ['precipitation', 'temperature', 'elevation', 'clusters']
        
        # Determinar qué fuente contiene los clusters para FiLM
        for i, x in enumerate(self.X_list):
            if 'clusters' in x:
                self.cluster_idx = i
                break

    def __len__(self):
        return len(self.X_list[0])
    
    def __getitem__(self, idx):
        # Combinar todas las fuentes en un solo tensor
        # Cada fuente: [batch_size, input_window, height, width]
        batch_inputs = []
        for x in self.X_list:
            # Tomar ventana completa para esta característica
            feature = x[idx]
            batch_inputs.append(feature)
        
        # Concatenar en dimensión de canal: [input_window, num_sources, height, width]
        combined_input = torch.cat([x.unsqueeze(1) for x in batch_inputs], dim=1)
        
        # Target: [output_horizon, height, width]
        target = self.y[idx]
        
        # Extraer el índice de cluster si está disponible
        if self.cluster_idx >= 0:
            # Tomar el primer índice de tiempo y la moda de los clusters en el mapa
            cluster_map = batch_inputs[self.cluster_idx][0]
            cluster_idx = int(torch.mode(cluster_map.flatten())[0])
        else:
            # Si no hay datos de cluster, usar 0 como fallback
            cluster_idx = 0
        
        return combined_input, target, cluster_idx

print("Definiendo modelos y dataset...")

# Inicializar el dataset
train_dataset = MultiSourceDataset(X_train, y_train, seq_length=OUTPUT_HORIZON)
val_dataset = MultiSourceDataset(X_val, y_val, seq_length=OUTPUT_HORIZON)

# Parámetros de modelo
if isinstance(X_train, list) and len(X_train) > 0:
    combined_channels = sum(x.shape[1] for x in X_train[0])
    print(f"Canales de entrada combinados: {combined_channels}")
else:
    combined_channels = X_train.shape[1]
    print(f"Canales de entrada: {combined_channels}")

# Definir cluster_ids basados en los datos disponibles
# Si tenemos datos de clusters, obtener valores únicos, sino usar un valor predeterminado
cluster_ids = list(range(10))  # Default: suponemos 10 clusters
# Definir target_shape basado en los datos
if isinstance(X_train, list) and len(X_train) > 0:
    # Si X_train es una lista, tomar las dimensiones del primer elemento
    if len(X_train[0].shape) >= 3:
        target_shape = X_train[0].shape[-2:]  # Últimas dos dimensiones (altura, anchura)
    else:
        # Dimensiones por defecto si no podemos determinarlas
        target_shape = (61, 65)
else:
    # Si X_train no es una lista, tomar sus dimensiones directamente
    if len(X_train.shape) >= 3:
        target_shape = X_train.shape[-2:]
    else:
        target_shape = (61, 65)

print(f"Target shape para la salida del modelo: {target_shape}")

# Instanciar modelos
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
convbigru_model = TopoClus_CEEMDAN_TVF_AFC_ConvBiGRU_AE(
    combined_channels, hidden_dim, n_clusters, OUTPUT_HORIZON
).to(device)

convbilstm_model = TopoClus_CEEMDAN_TVF_AFC_ConvBiLSTM_AE(
    combined_channels, hidden_dim, n_clusters, OUTPUT_HORIZON
).to(device)

def calculate_spatial_metrics(y_true, y_pred):
    """
    Calcula métricas espaciales entre valores verdaderos y predichos.
    """
    # Implementación pendiente
    pass

# Función para mostrar progreso sin borrar salidas previas
def plot_progress_without_clearing(train_losses, val_losses, best_val_loss=None):
    """
    Plotea el progreso del entrenamiento sin borrar la salida anterior.
    Similar a plot_progress pero sin el clear_output().
    """
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    if best_val_loss is not None:
        plt.axhline(y=best_val_loss, color='r', linestyle='--', label=f'Best: {best_val_loss:.2f}')
    plt.title(f'Loss vs. Epochs (Current: {val_losses[-1]:.2f})')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(alpha=0.3)
    
    plt.subplot(1, 2, 2)
    rel_loss = [l/train_losses[0] for l in train_losses]
    rel_val_loss = [l/val_losses[0] for l in val_losses]
    plt.plot(rel_loss, label='Train')
    plt.plot(rel_val_loss, label='Val')
    plt.title(f'Relative Loss (% of initial loss)')
    plt.xlabel('Epoch')
    plt.ylabel('Relative Loss')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

def train_hybrid_model(name, model, train_loader, val_loader, epochs=100, patience=20):
    """
    Entrena un modelo híbrido con optimizaciones avanzadas y realiza evaluación
    """
    print(f"\n{'='*30}")
    print(f"ENTRENAMIENTO DE {name}")
    print(f"{'='*30}")
    
    device = next(model.parameters()).device
    
    # Importar tqdm si no está disponible
    try:
        from tqdm import tqdm
    except ImportError:
        # Definir una versión simple si no está instalado
        def tqdm(iterable, **kwargs):
            print(kwargs.get('desc', ''))
            return iterable
    
    # Optimizador y función de pérdida
    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=patience//2, verbose=True
    )
    
    # Mixed precision
    scaler = GradScaler()
    
    # Tracking de métricas
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    epochs_no_improve = 0
    current_lr = optimizer.param_groups[0]['lr']  # Guardar LR inicial
    
    # Para guardar mejor modelo
    import time
    from datetime import datetime
    start_time = time.time()
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_name = name  # Asegurar que tenemos model_name definido
    save_dir = MODELS_OUTPUT  # Usar el directorio de modelos definido globalmente
    os.makedirs(save_dir, exist_ok=True)
    model_path = save_dir / f'{name}_{timestamp}.pt'
    
    for epoch in range(epochs):
        epoch_start = time.time()
        
        # ===== ENTRENAMIENTO =====
        model.train()
        train_loss = 0
        batch_metrics = []
        
        # Barra de progreso para entrenamiento
        train_progress = tqdm(train_loader, desc=f"Época {epoch+1}/{epochs} [Train]", leave=False)
        
        for inputs, targets in train_progress:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            
            # Asegurar compatibilidad de dimensiones
            if len(outputs.shape) == 5 and len(targets.shape) <= 4:
                outputs = outputs.squeeze(2)  # eliminar dim C si es 1
                
            if len(outputs.shape) != len(targets.shape):
                if len(outputs.shape) == 5 and len(targets.shape) == 3:
                    targets = targets.unsqueeze(1).unsqueeze(2).repeat(1, outputs.shape[1], 1, 1, 1)
                elif len(outputs.shape) == 5 and len(targets.shape) == 4:
                    targets = targets.unsqueeze(2)
            
            loss = criterion(outputs, targets)
            loss.backward()
            
            # Gradient clipping para estabilidad
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            # Actualizar barra de progreso con pérdida actual
            train_progress.set_postfix(loss=f"{loss.item():.4f}")
            batch_metrics.append(loss.item())
        
        train_loss = np.mean(batch_metrics)
        train_losses.append(train_loss)
        
        # ===== VALIDACIÓN =====
        model.eval()
        val_loss = 0
        val_outputs = []
        val_targets = []
        
        # Barra de progreso para validación
        val_progress = tqdm(val_loader, desc=f"Época {epoch+1}/{epochs} [Val]", leave=False)
        
        with torch.no_grad():
            for inputs, targets in val_progress:
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                outputs = model(inputs)
                
                # Ajustar dimensiones si es necesario
                if len(outputs.shape) == 5 and len(targets.shape) <= 4:
                    outputs = outputs.squeeze(2)
                
                if len(outputs.shape) != len(targets.shape):
                    if len(outputs.shape) == 5 and len(targets.shape) == 3:
                        targets = targets.unsqueeze(1).unsqueeze(2).repeat(1, outputs.shape[1], 1, 1, 1)
                    elif len(outputs.shape) == 5 and len(targets.shape) == 4:
                        targets = targets.unsqueeze(2)
                
                loss = criterion(outputs, targets)
                val_progress.set_postfix(loss=f"{loss.item():.4f}")
                val_loss += loss.item()
                
                # Guardar para métricas
                val_outputs.append(outputs.cpu())
                val_targets.append(targets.cpu())
        
        # Calcular pérdida promedio
        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        # Calcular tiempo de la época
        epoch_time = time.time() - epoch_start
        
        # Actualizar scheduler
        scheduler.step(avg_val_loss)
        
        # Comprobar si el LR ha cambiado
        new_lr = optimizer.param_groups[0]['lr']
        lr_updated = new_lr != current_lr
        old_lr = current_lr
        current_lr = new_lr  # Actualizar para próxima comparación
        
        # Mostrar gráfico de progreso cada 5 épocas o en la última
        if epoch % 5 == 0 or epoch == epochs - 1 or epochs_no_improve == patience:
            try:
                plot_progress_without_clearing(train_losses, val_losses, best_val_loss)
            except Exception as e:
                print(f"Error al mostrar gráfico: {str(e)}")
        
        # Imprimir resumen de la época
        print(f"\n📊 Época {epoch+1}/{epochs} completada en {epoch_time:.1f}s")
        print(f"   Train Loss: {train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | " + 
              (f"LR reducido: {old_lr:.6f} → {new_lr:.6f}" if lr_updated else f"LR: {new_lr:.6f}"))
        
        # Early stopping y guardado del mejor modelo
        if avg_val_loss < best_val_loss:
            improvement = (best_val_loss - avg_val_loss) / best_val_loss * 100 if best_val_loss != float('inf') else 100
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            
            # Guardar mejor modelo
            save_model(model, model_path, epoch, avg_val_loss)
            print(f"   ⭐ ¡Nuevo mejor modelo! Mejora: {improvement:.2f}%")
            
            # También guardar checkpoint específico de esta época
            epoch_path = save_dir / f'checkpoint_{model_name}_epoch_{epoch+1}.pth'
            save_model(model, epoch_path, epoch, avg_val_loss)
        else:
            epochs_no_improve += 1
            print(f"   ❌ Sin mejora durante {epochs_no_improve}/{patience} épocas. Mejor: {best_val_loss:.4f}")
            
            # Guardar checkpoint regular cada 10 épocas
            if epoch % 10 == 0:
                checkpoint_path = save_dir / f'regular_checkpoint_{model_name}_epoch_{epoch+1}.pth'
                save_model(model, checkpoint_path, epoch, avg_val_loss)
                
            if epochs_no_improve == patience:
                print(f"\n⚠️ Early stopping activado después de {patience} épocas sin mejora")
                break
    
    # Tiempo total de entrenamiento
    total_time = time.time() - start_time
    hours, rem = divmod(total_time, 3600)
    minutes, seconds = divmod(rem, 60)
    
    print(f"\n{'='*80}")
    print(f"ENTRENAMIENTO FINALIZADO: {model_name}")
    print(f"Tiempo total: {int(hours)}h {int(minutes)}m {seconds:.2f}s")
    print(f"Mejor pérdida de validación: {best_val_loss:.4f}")
    print(f"Modelos guardados en: {save_dir}")
    print(f"{'='*80}")
    
    # Cargar el mejor modelo
    try:
        checkpoint = torch.load(model_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"✅ Mejor modelo cargado de la época {checkpoint['epoch']+1}")
    except Exception as e:
        print(f"❌ Error al cargar el mejor modelo: {str(e)}")
    
    # Tiempo total de entrenamiento
    total_time = time.time() - start_time
    hours, rem = divmod(total_time, 3600)
    minutes, seconds = divmod(rem, 60)
    
    print(f"\n{'='*80}")
    print(f"ENTRENAMIENTO FINALIZADO: {model_name}")
    print(f"Tiempo total: {int(hours)}h {int(minutes)}m {seconds:.2f}s")
    print(f"Mejor pérdida de validación: {best_val_loss:.4f}")
    print(f"Modelos guardados en: {save_dir}")
    print(f"{'='*80}")
    
    return model, train_losses, val_losses

# Función para verificar si un modelo existe y mostrar información sobre él
def check_model_exists(model_path):
    """Verifica si un modelo existe y muestra información sobre él."""
    if os.path.exists(model_path):
        try:
            checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
            print(f"✅ Modelo encontrado en: {model_path}")
            if isinstance(checkpoint, dict) and 'epoch' in checkpoint:
                print(f"   Guardado en época: {checkpoint['epoch']+1}")
                print(f"   Pérdida validación: {checkpoint['val_loss']:.4f}")
            else:
                print("   (Formato antiguo - solo state_dict)")
            return True
        except Exception as e:
            print(f"❌ Error al cargar modelo {model_path}: {str(e)}")
            return False
    else:
        print(f"❌ Modelo no encontrado: {model_path}")
        return False

# Función para visualizar métricas de entrenamiento almacenadas
def visualize_training_metrics(train_losses, val_losses, model_name="modelo"):
    """
    Visualiza las métricas de entrenamiento con gráficos detallados y estadísticas.
    """
    plt.figure(figsize=(16, 12))
    
    # 1. Curva de pérdida básica
    plt.subplot(2, 2, 1)
    plt.plot(train_losses, label='Entrenamiento', color='blue', linestyle='-', marker='.', alpha=0.7)
    plt.plot(val_losses, label='Validación', color='red', linestyle='-', marker='.', alpha=0.7)
    
    best_val_idx = np.argmin(val_losses)
    best_val_loss = val_losses[best_val_idx]
    plt.axvline(x=best_val_idx, color='green', linestyle='--', alpha=0.7, 
                label=f'Mejor época: {best_val_idx+1}')
    plt.axhline(y=best_val_loss, color='green', linestyle=':', alpha=0.7)
    
    plt.title(f'Curva de Aprendizaje - {model_name}', fontsize=14)
    plt.xlabel('Época', fontsize=12)
    plt.ylabel('Pérdida', fontsize=12)
    plt.legend(loc='upper right')
    plt.grid(alpha=0.3)
    
    # 2. Pérdida relativa (%)
    plt.subplot(2, 2, 2)
    rel_train = [t/train_losses[0]*100 for t in train_losses]
    rel_val = [v/val_losses[0]*100 for v in val_losses]
    
    plt.plot(rel_train, label='Entrenamiento', color='blue', alpha=0.7)
    plt.plot(rel_val, label='Validación', color='red', alpha=0.7)
    plt.axvline(x=best_val_idx, color='green', linestyle='--', alpha=0.7)
    
    plt.title(f'Pérdida Relativa (% del valor inicial)', fontsize=14)
    plt.xlabel('Época', fontsize=12)
    plt.ylabel('Porcentaje (%)', fontsize=12)
    plt.legend()
    plt.grid(alpha=0.3)
    
    # 3. Diferencia Train-Val (sobreajuste)
    plt.subplot(2, 2, 3)
    diff = [t-v for t, v in zip(train_losses, val_losses)]
    
    plt.plot(diff, color='purple', alpha=0.7)
    plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    plt.axvline(x=best_val_idx, color='green', linestyle='--', alpha=0.7)
    
    # Sombreado para zona de sobreajuste potencial
    plt.fill_between(range(len(diff)), [0]*len(diff), diff, 
                     where=[d < 0 for d in diff], color='red', alpha=0.2,
                     label='Posible subajuste')
    plt.fill_between(range(len(diff)), [0]*len(diff), diff, 
                     where=[d > 0 for d in diff], color='orange', alpha=0.2,
                     label='Posible sobreajuste')
    
    plt.title('Diferencia Train-Validación (Indicador de Sobreajuste)', fontsize=14)
    plt.xlabel('Época', fontsize=12)
    plt.ylabel('Train Loss - Val Loss', fontsize=12)
    plt.legend()
    plt.grid(alpha=0.3)
    
    # 4. Velocidad de convergencia (derivada de la pérdida)
    plt.subplot(2, 2, 4)
    if len(val_losses) > 1:
        val_improvement = [val_losses[i-1] - val_losses[i] for i in range(1, len(val_losses))]
        improving = [i > 0 for i in val_improvement]
        colors = ['green' if imp else 'red' for imp in improving]
        
        plt.bar(range(1, len(val_losses)), val_improvement, color=colors, alpha=0.7)
        plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
        
        if best_val_idx > 0:
            plt.axvline(x=best_val_idx, color='green', linestyle='--', alpha=0.7)
        
        plt.title('Velocidad de Mejora por Época', fontsize=14)
        plt.xlabel('Época', fontsize=12)
        plt.ylabel('Mejora (reducción de pérdida)', fontsize=12)
        plt.grid(alpha=0.3)
    
    plt.tight_layout()
    
    # Estadísticas adicionales
    print(f"\n{'='*40} ESTADÍSTICAS DE ENTRENAMIENTO {'='*40}")
    print(f"Modelo: {model_name}")
    print(f"Total de épocas: {len(train_losses)}")
    print(f"Mejor época: {best_val_idx+1}")
    print(f"Mejor pérdida validación: {best_val_loss:.4f}")
    print(f"Pérdida inicial (val): {val_losses[0]:.4f}")
    print(f"Pérdida final (val): {val_losses[-1]:.4f}")
    print(f"Mejora total: {(1 - best_val_loss/val_losses[0])*100:.2f}%")
    
    # Calcular tendencias
    last_epochs = min(10, len(val_losses))
    if last_epochs > 1:
        recent_trend = val_losses[-last_epochs:][0] - val_losses[-1]
        print(f"Tendencia últimas {last_epochs} épocas: {recent_trend:.4f} " +
              ("📉 mejorando" if recent_trend > 0 else "📈 empeorando"))
    
    print(f"{'='*100}")
    
    return plt

# Reemplazar el código de entrenamiento con la versión mejorada
print("\n" + "="*80)
print("ENTRENAMIENTO DE MODELOS CON VISUALIZACIÓN MEJORADA")
print("="*80)

# Verificar que los modelos estén listos
if models_ready:
    try:
        # Verificar directorios de salida y rutas de modelos
        print("\n📂 Configuración de directorios y rutas:")
        print(f"Directorio de salida: {MODELS_OUTPUT}")
        
        # Comprobar si hay modelos guardados previamente
        convbigru_output_path = MODELS_OUTPUT / 'convbigru_ae_model.pth'
        convbilstm_output_path = MODELS_OUTPUT / 'convbilstm_ae_model.pth'
        
        print("\n🔍 Verificando modelos guardados previamente:")
        bigru_exists = check_model_exists(convbigru_output_path)
        bilstm_exists = check_model_exists(convbilstm_output_path)
        
        # Iniciar entrenamiento con seguimiento detallado
        if not bigru_exists or input("¿Volver a entrenar ConvBiGRU-AE? (s/n): ").lower() == 's':
            print("\n" + "="*50)
            print("ENTRENAMIENTO DE CONVBIGRU_AE CON VISUALIZACIÓN MEJORADA")
            print("="*50)
            
            # Entrenar con versión mejorada
            convbigru_ae, train_losses_bigru, val_losses_bigru = improved_train_model(
                convbigru_ae, train_loader, val_loader, 
                criterion, optimizer_convbigru, scheduler_convbigru,
                num_epochs, patience, model_name="ConvBiGRU-AE"
            )
            
            # Guardar modelo final con mensaje claro
            torch.save({
                'model_state_dict': convbigru_ae.state_dict(),
                'optimizer_state_dict': optimizer_convbigru.state_dict(),
                'train_losses': train_losses_bigru,
                'val_losses': val_losses_bigru
            }, convbigru_output_path)
            print(f"\n✅ Modelo final ConvBiGRU-AE guardado en {convbigru_output_path}")
            
            # Visualizar métricas detalladas
            print("\n📊 Visualización detallada de métricas de ConvBiGRU-AE:")
            visualize_training_metrics(train_losses_bigru, val_losses_bigru, "ConvBiGRU-AE")
        
        if not bilstm_exists or input("¿Volver a entrenar ConvBiLSTM-AE? (s/n): ").lower() == 's':
            print("\n" + "="*50)
            print("ENTRENAMIENTO DE CONVBILSTM_AE CON VISUALIZACIÓN MEJORADA")
            print("="*50)
            
            # Entrenar con versión mejorada
            convbilstm_ae, train_losses_bilstm, val_losses_bilstm = improved_train_model(
                convbilstm_ae, train_loader, val_loader, 
                criterion, optimizer_convbilstm, scheduler_convbilstm,
                num_epochs, patience, model_name="ConvBiLSTM-AE"
            )
            
            # Guardar modelo final con mensaje claro
            torch.save({
                'model_state_dict': convbilstm_ae.state_dict(),
                'optimizer_state_dict': optimizer_convbilstm.state_dict(),
                'train_losses': train_losses_bilstm,
                'val_losses': val_losses_bilstm
            }, convbilstm_output_path)
            print(f"\n✅ Modelo final ConvBiLSTM-AE guardado en {convbilstm_output_path}")
            
            # Visualizar métricas detalladas
            print("\n📊 Visualización detallada de métricas de ConvBiLSTM-AE:")
            visualize_training_metrics(train_losses_bilstm, val_losses_bilstm, "ConvBiLSTM-AE")
        
        print("\n✅ Proceso de entrenamiento mejorado completado")
        
    except Exception as e:
        print(f"\n❌ Error durante el entrenamiento mejorado: {str(e)}")
        import traceback
        traceback.print_exc()
else:
    print("\n❌ Los modelos no están listos para entrenar. Verifica la configuración.")

# Función para analizar y visualizar modelos guardados
def analyze_saved_models(model_dir=MODELS_OUTPUT, pattern="*_ae_model.pth"):
    """
    Analiza y visualiza información sobre los modelos guardados
    """
    import glob
    import os
    from pathlib import Path
    
    # Encontrar todos los archivos que coinciden con el patrón
    model_files = list(Path(model_dir).glob(pattern))
    
    if not model_files:
        print(f"❌ No se encontraron modelos con el patrón '{pattern}' en {model_dir}")
        return
    
    print(f"\n📊 Modelos encontrados: {len(model_files)}")
    for i, model_path in enumerate(model_files, 1):
        print(f"\n{i}. {model_path.name}:")
        
        # Obtener información del archivo
        size_mb = os.path.getsize(model_path) / (1024*1024)
        modified_time = datetime.fromtimestamp(os.path.getmtime(model_path))
        
        print(f"   📁 Tamaño: {size_mb:.2f} MB")
        print(f"   🕒 Última modificación: {modified_time.strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Intentar cargar el modelo para obtener más información
        try:
            checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
            if isinstance(checkpoint, dict):
                print("   📋 Contenido del checkpoint:")
                for key, value in checkpoint.items():
                    if key == 'model_state_dict':
                        n_params = sum(p.numel() for p in value.values())
                        print(f"      - model_state_dict: {n_params:,} parámetros")
                    elif key == 'optimizer_state_dict':
                        print(f"      - optimizer_state_dict: incluido")
                    elif isinstance(value, list):
                        print(f"      - {key}: lista de {len(value)} elementos")
                    elif isinstance(value, (int, float)):
                        print(f"      - {key}: {value}")
                    else:
                        print(f"      - {key}: {type(value).__name__}")
                        
                # Si contiene historial de pérdidas, visualizarlo
                if 'train_losses' in checkpoint and 'val_losses' in checkpoint:
                    train_losses = checkpoint['train_losses']
                    val_losses = checkpoint['val_losses']
                    
                    plt.figure(figsize=(10, 6))
                    plt.plot(train_losses, label='Train')
                    plt.plot(val_losses, label='Validation')
                    plt.title(f'Historial de entrenamiento - {model_path.stem}')
                    plt.xlabel('Época')
                    plt.ylabel('Pérdida')
                    plt.legend()
                    plt.grid(alpha=0.3)
                    plt.show()
                    
                    print(f"      - Épocas entrenadas: {len(train_losses)}")
                    print(f"      - Pérdida inicial: {train_losses[0]:.4f} (train), {val_losses[0]:.4f} (val)")
                    print(f"      - Pérdida final: {train_losses[-1]:.4f} (train), {val_losses[-1]:.4f} (val)")
                    print(f"      - Mejor pérdida val: {min(val_losses):.4f} (época {np.argmin(val_losses)+1})")
        except Exception as e:
            print(f"   ❌ Error al analizar el modelo: {str(e)}")
    
    return model_files

# Ejecutar análisis de modelos guardados
print("\n" + "="*50)
print("ANÁLISIS DE MODELOS GUARDADOS")
print("="*50)
saved_models = analyze_saved_models()
# Ejecutar análisis de modelos guardados
print("\n" + "="*50)
print("ANÁLISIS DE MODELOS GUARDADOS")
print("="*50)
saved_models = analyze_saved_models()
print(f"\n📂 Directorio de modelos guardados: {MODELS_OUTPUT}")
